<a href="https://colab.research.google.com/github/mbarbetti/unifi-physics-lab3/blob/main/Fit_lifetime_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Misura della vita media del muone
Lo scopo di questo programma è misurare la vita media del muone facendo un fit hai dati raccolti in laboratorio
Il primo blocco qui sotto serve solo a importare (e installare quelli che non sono disponibili) alcuni pacchetti che ci sono necessari e in particolare Minuit, il programma di minimizzazione, che si trova nella libreria iminuit, che deve essere installata con pip perché non è presente tra le librerie disponibili di default in Colab.
L'ultima riga serve per ridefinire le dimensioni dei plots.

In [ ]:
import pandas as pd
from urllib.request import urlopen
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import poisson
from scipy.stats import chi2
!pip install -q iminuit
from iminuit import Minuit


plt.rcParams['figure.figsize'] = [16, 8]

Prima di tutto dovete **modificare il codice inserendo il nome del gruppo**, in modo che il programma trovi il file con i dati. Le istruzioni che seguono stampano l'header del file con alcune informazioni che ci serviranno in seguito. 

La prima operazione necessaria è importare i dati nel file scritto da LabView in formato pandas. Dopo questa operazione i nostri dati si trovano nell'oggetto chiamato **data** che è di tipo DataFrame (una tabella).
Alla fine si moltiplicano i valori TDC per 1e09 per trasformarli in ns. 

In [ ]:
## INSERIRE NUMERO DEL GRUPPO
group = "M2"   # esempio

### SET THE VALUE OF A AND B
a = 0.97693 #ns/a.u.
b = -90.14805 #ns
    
header = urlopen (f"https://raw.githubusercontent.com/mbarbetti/unifi-physics-lab3/main/data/2022/data_{group}.txt")
for i, line in enumerate(header):
  print (line.decode('utf-8'))
  if i == 5: break

print ('Calibration factors:')
print ('    a = {:.5f} ns/a.u.' . format (a))
print ('    b = {:.5f} ns' . format (b))
print ('\nt_corr = ({:.5f}) * t_mis + ({:.5f}) ns' . format (a, b))


In [ ]:
file_path = f"https://raw.githubusercontent.com/mbarbetti/unifi-physics-lab3/main/data/2022/data_{group}.txt"
data = pd.read_csv (file_path, header = 4, delim_whitespace = True).drop([0])
data['QDC1'] = -data['QDC1'] * 1e9
data['QDC2'] = -data['QDC2'] * 1e9
data['TDC'] = data['TDC'] * 1e9
print(data)   # dataframe
print(min(data['QDC1'])," ",min(data['QDC1']))

Qui si riempiono degli istogrammi bi- e uni-dimensionali con i valori del QDC1 e QDC2 (rispettivamente muone ed **elettrone**). Le unità sono arbitrarie e sono moltiplicate per 1e09 per comodità di visualizzazione. Queste distribuzioni servono per selezionare gli eventi di fisica rispetto a quelli di calibrazione: guardando allo scatter plot **dovete decidere dove mettere i tagli per selezionare gli eventi corrispondenti ai decadimenti dei muoni**

In [ ]:
### Plot qdc values
### SET THE QDC RANGES FOR MUON DECAY SIGNAL 
qdc1_min = 3.5
qdc1_max = 10 
qdc2_min = 3.4
qdc2_max = 10

data_muon = data.query (f"(QDC1 > {qdc1_min}) & (QDC1 < {qdc1_max}) & (QDC2 > {qdc2_min}) & (QDC2 < {qdc2_max})")
qdc1 = data_muon['QDC1']
qdc2 = data_muon['QDC2']
qdc1_all = data['QDC1']
qdc2_all = data['QDC2']

plt.xlabel ("QDC1")
plt.ylabel ("QDC2")
plt.plot ([0,10], [qdc2_min, qdc2_min], color = "red", lw = 2)
plt.plot ([qdc1_min, qdc1_min], [0,10], color = "red", lw = 2)
plt.scatter (qdc1_all, qdc2_all, s=1)
plt.axis ([2,10,2,8])
plt.show()

nbin = 100

plt.xlabel ("QDC1")
plt.ylabel ("Entries")
plt.hist (qdc1_all, bins=nbin, range = (2,10), alpha = 0.75, label='QDC1')
plt.hist (qdc1, bins=nbin, range = (2,10), alpha = 0.75, label='QDC1')
plt.yscale("log")
plt.show()

plt.xlabel ("QDC2")
plt.ylabel ("Entries")
plt.hist (qdc2_all, bins=nbin, range = (2,10), alpha = 0.75, label='QDC2')
plt.hist (qdc2, bins=nbin, range = (2,10), alpha = 0.75, label='QDC2')
plt.yscale("log")
plt.show()

Si riempie adesso l'istogramma con i valori dei tempi. Questo istogramma è poi usato per il fit, quindi si deve prestare attenzione a definire **tmin** e **tmax** in modo da eliminare eventuali valori non fisici e le regioni vicine ai valori limite di accettanza della nostra misura.  
Scegliete il numero di bin **nbins** e di conseguenza la larghezza del bin **delta_t**. La larghezza del bin deve essere scelta in modo che la densità di probabilità non cambi significativamente nell'intervallo del bin, ma anche che sia abbastanza grande perché si possa applicare il test del $\chi^2$ di Pearson (ovvero i conteggi devono essere > 10). 
Per ingrandire una zona dell'istogramma potete modificate gli estremi dell'asse x con **plt.xlim(right,left)** e la scala verticale può essere impostata in modo logaritmico con **plt.yscale("log")**


In [ ]:
### Prepare and fill histogram of time intervals
### SET TMIN, TMAX AND NBINS
tmin = 100
tmax = 10000
nbins = 200

delta_t = (tmax - tmin) / nbins

### Apply calibration factors

tdc = a*data_muon['TDC'].to_numpy() + b

histo_times = np.histogram(tdc, bins = nbins, range = (tmin, tmax))
#print(histo_times)

plt.xlabel('Time interval [ns]')
plt.ylabel('Number of events')
plt.hist (tdc, bins = nbins, range = (tmin, tmax), label='Measured time intervals')
plt.yscale("linear") #linear or log
plt.show()


Si riempiono adesso i vettori con i valori di x e y che saranno usati per il fit

In [ ]:
### Prepare data for the fit
# histo_times #print the bin values and edges 
### Super cool way to get bin centers
x_values = (histo_times[1][1:] + histo_times[1][:-1]) / 2
y_values = histo_times[0]
#bin_edges = histo_times[1].tolist()
print(x_values)
print(y_values)
print (delta_t)

Si costruisce la negative log-likelihood (NLL) considerando il numero di eventi in ogni bin come il risultato di una distribuzione di poisson con valore medio dato da $ \nu_i = \nu_{tot}\int_{x_i}^{x_{i+1}} \frac{1}{\tau}\exp(-\frac{x}{\tau}) + A\Delta t \approx \nu_{tot}\frac{1}{\tau}\exp(-\frac{x}{\tau})\Delta t + A\Delta t $

L'ultimo termine $A\Delta t$ rappresenta il numero di coincidenze casuali in ogni bin, con il parametro $ A = R^2\cdot T_M \cdot 10^{-9} \, [ns^{-1}]$, dove $ R $ è il ritmo di arrivo dei muoni in *Hz* e $ T_M $ il tempo totale di misura in *s*. Il fattore $10^{-9}$ tiene conto che $\Delta t$ è espresso in *ns*. 

Il fattore $A$ è posto inizialmente a 0 ed è fissato nel fit dall'opzione di Minuit *fix_A = True*. Successivamente potete provare a lasciarlo libero nel fit o a fissarlo al valore calcolato tramite la formula qui sopra. 
Al posto della semplice esponenziale si può usare la funzione *fullExp()* che tiene conto dell'assorbimento dei muoni negativi.  
Le varie likelihood che possono essere usate nel fit sono: 

1) *NLLbin()*: binned extended maximum likelihood con la funzione *simpleExp()* per i decadimenti, che non tiene conto dell'assorbimento dei muoni

2) *NLLass()*: come sopra ma con *fullExp()* per i decadimenti, che tiene conto dell'assordimento


In [ ]:
### Build Negative Log Likelihood (NLL) to minimize

def simpleExp(t, nu, tau):

    return nu*(np.exp(-t/tau)/tau)

def fullExp(t, nu, tau):

    r = 1.27 
    lass = 38.8e-6

    return (nu/tau)/(1+r)*(r+np.exp(-lass*t))*np.exp(-t/tau)

def NLLbin(nu, tau, A):
    
    nlogL = 0

    for i, count in enumerate(y_values):
        nlogL = nlogL - poisson.logpmf(count, (simpleExp(x_values[i], nu, tau) + A)*delta_t )
    
    return nlogL


def NLLass(nu, tau, A):
    
    nlogL = 0

    for i, count in enumerate(y_values):
        nlogL = nlogL - poisson.logpmf(count, (fullExp(x_values[i], nu, tau) + A)*delta_t)
    
    return nlogL


Adesso si inizializza *Minuit* passando la funzione da minimizzare e il valore dei vari parametri. Successivamente si chiama *migrad()* per minimizzare NLL e poi *hesse()* per ri-calcolare più precisamente la matrice di covarianza e infine *minos()* per calcolare gli errori. Per chi vuole approfondire, la documentazione di *Minuit* è disponibile al link https://iminuit.readthedocs.io/en/stable/

**Il primo argomento è la funzione di fit:** si possono indicare *NLLbin*, *NLLass*. Fate riferimento al blocco precedente per la descrizione.

**I parametri possono essere lasciati liberi o fissati nel fit** impostando m.fixed['nome del parametro'] rispettivamente su *False* o *True*. Per esempio il parametro *A* è inizialmente fissato.  

In [ ]:
### Call Migrad to find minimum of the NLL
### SET NLL FUNCTION TYPE AS NLLbin or NLLass
NLL = NLLass
m = Minuit(NLL, nu=4000, tau=2200., A = 0.0)
m.errordef = Minuit.LIKELIHOOD     # == 0.5
m.fixed['A'] = True
m.migrad()
m.hesse()
m.minos()
m.migrad()

Stampiamo i parametri ottenuti dal fit in una forma più leggibile

In [ ]:
### Get fit results and errors from Minuit
nu_fit = m.values['nu']
tau_fit = m.values['tau']
A_fit = m.values['A']

nu_fit_err = m.errors['nu']
tau_fit_err = m.errors['tau']
A_fit_err = m.errors['A']

### Print the results
if m.fixed['A']:
  print(f'Parameters from fit:\n\tnu_tot = {nu_fit:.0f} +- {nu_fit_err:.0f} \n\tTau = {tau_fit:.2f} +- {tau_fit_err:.2f} ns\n\tA = {A_fit:.3f} 1/ns  (fixed)')
else:
  print(f'Parameters from fit:\n\tnu_tot = {nu_fit:.0f} +- {nu_fit_err:.0f} ns^-1\n\tTau = {tau_fit:.2f} +- {tau_fit_err:.2f} ns\n\tA = {A_fit:.3f} +- {A_fit_err:.3f} 1/ns')

Confrontiamo il risultato ottenuto con la distribuzione nei dati. Possiamo anche calcolare il $\chi^2$. 

**Si noti che il numero di gradi di libertà *ndf* è definito tenendo conto del numero di parametri liberi nel fit che si ottiene con m.nfit**


In [ ]:
### Plot both the histograms as well as the fitted distribution

if NLL != NLLass:
  pdf = simpleExp
else:
  pdf = fullExp

t_plot = np.arange (0, 10000, 10)
y_plot = (pdf(t_plot, nu_fit, tau_fit) + A_fit)*delta_t
plt.plot(t_plot, y_plot, label='Fitted distribution', color = "red", zorder = 1, lw = 2)

y_errors = np.sqrt(y_values)
plt.errorbar(x_values, y_values, yerr=y_errors, elinewidth=2, linewidth=0, color='blue', markerfacecolor='blue', markersize=4, markeredgewidth=3, marker='v', label='Data points', zorder = 0)
plt.legend()
plt.show()

### Calculate chi2 and print fit results
y_fit = (pdf(x_values, nu_fit, tau_fit) + A_fit)*delta_t
residuals = y_values - y_fit
squares = np.square(residuals)/y_fit
chi2fit = squares.sum()   ## chi2 value
ndf = len(squares) - m.nfit ## number of degree of freedom
p_val = 1 - chi2.cdf (chi2fit, ndf)

print(f'Chi square of the fitted distribution:\n\tchi2 = {chi2fit:.4f}\n\tndf = {ndf:.4f}\n\tp-value = {p_val:.4f}')


Usiamo la funzione *draw_mnprofile()* per disegnare il profilo della negative log-likelihood (NLL) in un intorno del minimo al variare di $\tau$. 

Si noti che per ogni valore di tau vengono ricalcolati i valori degli altri parametri che minimizzano la NLL: questo equivale a considerare la densità di probabilità marginale per il parametro per cui si disegna il profilo.

La banda mostrata corrisponde ad una variazione di $\pm 1 \sigma$

In [ ]:
m.draw_mnprofile('tau')
plt.show()

Se il parametro *A* non è fissato, usiamo la funzione *draw_mncontour()* per disegnare le linee di livello corrispondenti ad un livello di confidenza pari al 39% e al 68%. Per curiosità potete provare a vedere come viene il contorno anche per *nu* verso *tau* o verso *A*. 

**Sapreste dire a quale variazione del valore della negative log-likelihood corrispondono queste linee di livello?**

In [ ]:
### Ask Minuit for the 2D contour at CL = 39% and CL = 68% if A is not fixed
if not m.fixed['A']:  
  m.draw_mncontour('tau','A', cl=[0.39,0.68])

**Passi successivi**

1) Adesso provate a lasciare libero il parametro A. Che valori ottenete? Come cambia il chi2? E l'errore su *tau*? Provate a guardare l'ellisse di correlazione tra *A* e *tau*: sapete spiegarvi il risultato? 

2) Provate adesso a tenere conto dell'effetto dell'assorbimento dei muoni negativi. Il valore trovato è compatibile con quello noto in letteratura $\tau = 2197$ ns? 

3) Provate infine a calcolare A utilizzando il valore di $R$ misurato in laboratorio. 

Utilizzate il tempo $T_M$ della misura che è scritto nell'header del file con i dati e che abbiamo stampato all'inizio. Ricordate di sottrarvi un numero di secondi pari numero di eventi raccolti per tener conto del tempo morto dovuto all'acquiaizione. Questo tempo morto può essere considerato come l'errore sistematico du $T_M$.

Dopo tutte queste considerazioni qual è l'errore sulla vostra stima di A? 
Il risultato ottenuto è compatibile con quello ottenuto dal fit al punto precente (con NLLass)?

Per tener conto di questa incertezza ripetete il fit (sempre con NLLass) modificando A di più o meno l'errore. La variazione nel risultato del fit per $\tau$ è l'errore sistematico dovuto ad A. Quanto è importante se confrontato con l'errore statistico del fit? 

Quanto vale l'errore totale sulla misura? Confrontatelo con quello che avete ottenuto al punto 2) e commentate la differenza. Per una discussione finale e per tirare le vostre conclusioni tenete anche conto dei valori del chi2 (e dei relativi p-valori) che avete ottenuto
